##**1.必要なライブラリのインストール**

In [1]:
#OpenAIライブラリ(バージョン固定)
!pip install openai==0.28
import openai

#Whisperライブラリ
!pip install git+https://github.com/openai/whisper.git -q
import whisper
model = whisper.load_model("large-v2")

#フォルダ・ファイル操作などのためのライブラリ
import os
import glob
import time
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


100%|█████████████████████████████████████| 2.87G/2.87G [01:27<00:00, 35.5MiB/s]


## **2.OpenAIのAPIキーの入力と動画ファイルのダウンロード**

In [2]:
# YouTubeからファイルをダウンロードするライブラリをインストール
!python -m pip install -U yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.1 MB/s eta 0:00:00


In [3]:
openai.api_key = "" #@param {type:"string"}

In [4]:
# YouTube動画から音声ファイルをダウンロード
# 「mapry　デモ会　[ 林業 ICT化 小規模 低コスト LiDAR 測量アプリ ]　ー北都留森林組合」
# 上記の動画を使用させていただきました。この動画から「mapry.mp3」を出力する
!yt-dlp -x --audio-format mp3 https://youtu.be/wpPJdNe9Uuw?si=PAtZfs1e9xCDHEwb -o "mapry.mp3"

meeting_file_name = "mapry.mp3"
meeting_file_path = "/content/" + meeting_file_name

[youtube] Extracting URL: https://youtu.be/wpPJdNe9Uuw?si=PAtZfs1e9xCDHEwb
[youtube] wpPJdNe9Uuw: Downloading webpage
[youtube] wpPJdNe9Uuw: Downloading ios player API JSON
[youtube] wpPJdNe9Uuw: Downloading android player API JSON
[youtube] wpPJdNe9Uuw: Downloading m3u8 information
[info] wpPJdNe9Uuw: Downloading 1 format(s): 251
[download] Destination: mapry.webm
[download] 100% of    5.84MiB in 00:00:00 at 13.67MiB/s
[ExtractAudio] Destination: mapry.mp3
Deleting original file mapry.webm (pass -k to keep)


## **3.メイン処理**

In [5]:
#音声ファイルの読み込み

# カレントディレクトリを取得
current_directory = os.getcwd()

# 探したいファイルの拡張子を定義（必要に応じて拡張子を追加または変更）
file_extensions = ["*.mp4", "*.avi", "*.mp3", "*.wav"]

# 各拡張子に対してファイルを検索し、リストに格納
waves = []
for extension in file_extensions:
    waves.extend(glob.glob(os.path.join(current_directory, extension)))

In [6]:
#文字起こしの結果をテキストとして保存
def create_textfile(_index,video_name):
    #名前の入力
    txt = str(video_name) + ".txt"
    results = model.transcribe(video_name, verbose=True, language="ja")
    with open(txt, mode="a") as f:
        f.write(f'{txt}\n')
        for index, _dict in enumerate(results["segments"]):
            f.write(f'{_dict["text"]}\n')
    return txt

#文字起こし実行、txtファイル作成
for index,video in enumerate(waves):
  create_textfile(str(index),video)

[00:00.000 --> 00:02.880] 今日はお集まりいただきありがとうございます
[00:02.880 --> 00:04.880] マプリーの中村と申します
[00:04.880 --> 00:07.720] 普段は兵庫県の丹波市といわれる
[00:07.720 --> 00:11.280] 神戸からも世の遠い山の方で
[00:11.280 --> 00:15.040] 森林管理ですとか測量のアプリケーションを作っています
[00:15.040 --> 00:17.160] 埋木調査は臨着でやられていたりですとか
[00:17.160 --> 00:19.680] 流行を測るのがテクスチャーによっていたりですとか
[00:19.680 --> 00:22.000] いろんなツールがあったと思うんですけれども
[00:22.000 --> 00:24.320] それを全て一つにまとめて
[00:24.320 --> 00:27.840] こういったiPadで森林管理、埋木調査ですとか
[00:27.840 --> 00:31.840] 作業道の延長等ですとか周囲測量ですとか
[00:31.840 --> 00:35.080] そういったところが全部一つでできるようなツールになっています
[00:35.080 --> 00:36.520] この4月からですね
[00:36.520 --> 00:39.120] 森林整備事業、森林保守事業の方でもですね
[00:39.120 --> 00:41.600] うちのサービスを使って周囲測量をするっていうところで
[00:41.600 --> 00:43.120] 認められた件も出てきまして
[00:43.120 --> 00:45.120] 現場の方が使いやすくて
[00:45.120 --> 00:48.320] 普段の業務が楽になるみたいな形で作っていますので
[00:48.320 --> 00:50.920] いろいろと見ていただければなと思っております
[00:50.920 --> 00:53.040] 具体的にはなんですけれども
[00:53.040 --> 00:54.480] iPhoneだったりですとか
[00:54.480 --> 00:55.920] iPadの上位機種
[00:55.920

In [7]:
# プロンプトの実行
def completion(text):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",  # または利用可能な最新のモデル
        prompt=text,
        max_tokens=1024,
        temperature=1.0,
        top_p=1.0,
        frequency_penalty=0.02,
    )
    return response['choices'][0]['text']

# サマリー作成用のプロンプト
def create_prompt(transcript):
    return f"""以下は、ある会議の書き起こしです。

{transcript}

この会議を要約してください。以下のような形式で書いてください。

- 会議の目的
- 会議の具体的な内容
- 会議の結果

サマリー:
"""

# 今後のアクション作成用のプロンプト
def create_prompt_act(transcript):
    return f"""以下は、ある会議の書き起こしです。

{transcript}

この会議の次にするアクションを作成してください。アクションの記述は以下のルールに従ってください。

・リスト形式で出力する (先頭は - を使う)
・簡潔に表現する

アクション:
"""
files = glob.glob("/content/*.txt")  # 適切なパスに修正してください

transcript = ""
for file in files:
    with open(file, "r") as f:
        content = f.read()
        transcript += content + "\n"

prompt = create_prompt(transcript)
summary = completion(prompt)  # APIキーを引数として渡す必要がない
act_prompt = create_prompt_act(transcript)
action = completion(act_prompt)  # APIキーを引数として渡す必要がない


## **文字起こしの内容を表示**

In [8]:
print(transcript)

/content/mapry.mp3.txt
今日はお集まりいただきありがとうございます
マプリーの中村と申します
普段は兵庫県の丹波市といわれる
神戸からも世の遠い山の方で
森林管理ですとか測量のアプリケーションを作っています
埋木調査は臨着でやられていたりですとか
流行を測るのがテクスチャーによっていたりですとか
いろんなツールがあったと思うんですけれども
それを全て一つにまとめて
こういったiPadで森林管理、埋木調査ですとか
作業道の延長等ですとか周囲測量ですとか
そういったところが全部一つでできるようなツールになっています
この4月からですね
森林整備事業、森林保守事業の方でもですね
うちのサービスを使って周囲測量をするっていうところで
認められた件も出てきまして
現場の方が使いやすくて
普段の業務が楽になるみたいな形で作っていますので
いろいろと見ていただければなと思っております
具体的にはなんですけれども
iPhoneだったりですとか
iPadの上位機種
工戸シリーズと呼ばれているシリーズにですね
レーザーを飛ばすような機能がつきました
今まで航空レーザーとかであれば
皆さんイメージが当たると思うんですけれども
ここから光を飛ばして
物体の形状ですとか
強行直径どれぐらいなのかだったりとか
実測レベルで測れるようなセンサーがつきましたので
林内を歩いていくっていうところは変わらないんですけれども
一本一本こうやって測りながら歩いていく
瞬時にもうそこで強行直径だったりですとか
流木が山の中でどこの位置にあるかっていうのも
抑えることができますので
そういったオフラインの環境でも動くように
作っているって感じでしょうか
皆さんスマートフォンだったりお使いであれば
普段のGoogleマップみたいなものに近いんですけれども
プラスアルファでいろんな機能がついていまして
今僕はGoogleマップの上に
シリーズの地図を重ねています
東高線とか見えるので
すごい便利かなと思っていたりですとか
もちろん接着物体図ですとか
岐阜図とかも表示させることができますので
この上で重ねて
これを事務所等でダウンロードしておいていただければ
山の中でも使えるような形で作っています
オフラインの環境でも
iPhone、iPadも積んでいるGPSがですね
かなり市場のものより精度

## **要約結果をマークダウン形式で出力**

In [9]:
from IPython.display import display, Markdown, Latex

display(Markdown("## Summary\n\n" + summary + "\n\n## Action\n\n" + action))

## Summary

- 会議の目的: マプリー中村氏が開催した会議では、森林管理や測量のアプリケーションについて紹介し、その優位性を説明することが目的であった。
- 会議の具体的な内容: マプリー中村氏は、普段は兵庫県に拠点を置き、森林管理や測量のアプリケーション開発を行っていると紹介した。その後、森林整備事業や森林保守事業などでそのアプリケーションが認められ、現場での作業を楽にすることができるという点を強調した。具体的なアプリケーションとして、iPadを使用した森林管理、埋木調査、作業道の延長などのツールを紹介し、これらをすべて一つのアプリケーションにまとめることで、作業の効率化が図られることを説明した。さらに、Googleマップの上にシリーズの地図を重ねることで、山の中でも使えるようにオフライン環境でも動作可能なアプリケーションであることを説明した。また、iPhoneやiPadの上位機種にレーザー機能が搭載されており、実測できるセンサーもあることを説明した。さらに、3Dスキャン機能もあり、木の形状や大きさを簡単に測定できることを説明した。また、標準値測定もアプリ内で行えるようにし、従来必要だったビニールテープなどの道具を不要にすることで作業の手間を大幅に減らせることを説明した。さらに、頂点が見える位置まで下がって行画の角度センサーで撮影することで精度を高めていることも説明した。
- 会議の結果: マプリー中村氏のアプリケーションが認められ、現場の作業をより効率的に行えることが確認され、参加者からも高い評価を得ることができた。

## Action

- マプリーのツールの詳細について調査する
- ツールの使い方を実際に試してみる
- 埋木調査や周囲測量など、どのような業務に使えるか考える
- ツールを使った作業のイメージを具体的にする
- ツールを導入した場合、どのようなメリットがあるか検証する
- Vertexさんを使用してみて、操作性や精度の確認をする
- 標準値の注釈を画面上で作る方法を研究する